In [1]:
import cv2

In [4]:
img = cv2.imread("/content/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_1344_1568_size224.png", cv2.IMREAD_GRAYSCALE)

In [5]:
img.shape

(224, 224)

In [6]:
from collections import defaultdict

In [7]:
from scipy.ndimage import label

In [8]:
import numpy as np

In [20]:
classes = defaultdict(lambda: {"counts": 0, "surface": 0})

In [21]:
for cl, s in zip(*np.unique(img, return_counts=True)):
    classes[cl]["surface"] = s

In [24]:
for cl in classes.keys():
    mask = np.zeros(img.shape)
    mask[img == cl] = 1
    _, num_objects = label(mask)
    classes[cl]["counts"] = num_objects

In [25]:
classes

defaultdict(<function __main__.<lambda>()>,
            {0: {'counts': 2, 'surface': 14567},
             1: {'counts': 2, 'surface': 6653},
             2: {'counts': 1, 'surface': 28956}})

In [58]:
def calc_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    classes = defaultdict(lambda: {"counts": 0, "surface": 0})

    for cl, s in zip(*np.unique(img, return_counts=True)):
        classes[int(cl)]["surface"] = int(s)

    for cl in classes.keys():
        mask = np.zeros(img.shape)
        mask[img == cl] = 1
        _, num_objects = label(mask)
        classes[int(cl)]["counts"] = int(num_objects)

    return dict(classes)

In [59]:
images = ["/content/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_1344_1568_size224.png",
          "/content/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_1120_3136_size224.png"] * 100

In [60]:
import json

In [63]:
def calc_images(images, file=None):
    result = []
    for p in images:
        result.append({"path": p, "classes": calc_image(p)})

    data = {"images": result}

    if file is not None:
        with open(file, "w") as f:
            json.dump(data, f, indent=2)

    return data

In [64]:
%time res = calc_images(images, file="/content/data.json")

CPU times: user 520 ms, sys: 1.39 ms, total: 521 ms
Wall time: 522 ms


In [65]:
%%writefile "/content/image_process.py"

import cv2
from collections import defaultdict
import numpy as np
from scipy.ndimage import label
import json


def calc_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    classes = defaultdict(lambda: {"counts": 0, "surface": 0})

    for cl, s in zip(*np.unique(img, return_counts=True)):
        classes[cl]["surface"] = s

    for cl in classes.keys():
        mask = np.zeros(img.shape)
        mask[img == cl] = 1
        _, num_objects = label(mask)
        classes[cl]["counts"] = num_objects

    return dict(classes)


def calc_images(images, file=None):
    result = []
    for p in images:
        result.append({"path": p, "classes": calc_image(p)})

    data = {"images": result}

    if file is not None:
        with open(file, "w") as f:
            json.dump(data, f, indent=2)

    return data

Writing /content/image_process.py
